# Part 1


* Your stakeholder only wants you to include information for movies based on the following specifications:

* Exclude any movie with missing values for genre or runtime

* Include only full-length movies (titleType = "movie").

* Include only fictional movies (not from documentary genre)

* Include only movies that were released 2000 - 2021 (include 2000 and 2021)

* Include only movies that were released in the United States

## Imports and Data

In [1]:
#imports

import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# getting urls

basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz' 

In [3]:
#storing urls

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep ='\t', low_memory = False)
ratings = pd.read_csv(ratings_url, sep = '\t', low_memory = False)


## Basics Data and Cleaning

In [4]:
# Exploring df

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
#Checking types

basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9244025 entries, 0 to 9244024
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 634.7+ MB


In [6]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64

In [7]:
# data dictionary indicates nans replaced with /N, replacing with np.nan
basics.replace({'\\N':np.nan}, inplace = True)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [8]:
#Dropping na columns
basics.dropna(inplace = True) 
basics.isna().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64

In [9]:
#checking for duplicate rows
basics.duplicated().sum()

0

In [10]:
#Changing start year dtype from object to int 
basics['startYear'] = basics['startYear'].astype(int)
basics = basics.loc[(basics['startYear'] >=2000) & (basics['startYear'] <=2022)]

In [11]:
# get data type of column 'startYear'
dataType = basics.dtypes['startYear']
print('Data type of each column startYear in the Dataframe :')
print(dataType)

Data type of each column startYear in the Dataframe :
int32


In [12]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
108529,tt0111056,tvSeries,Gensomaden Saiyuki,Gensomaden Saiyuki,0,2000,2001,23,"Action,Adventure,Animation"
137803,tt0142032,tvMiniSeries,Dune,Dune,0,2000,2000,265,"Adventure,Drama,Sci-Fi"
153524,tt0158466,tvMiniSeries,Anne of Green Gables: The Continuing Story,Anne of Green Gables: The Continuing Story,0,2000,2000,185,"Drama,Family,Romance"
155717,tt0160904,tvSeries,MI-5,Spooks,0,2002,2011,60,"Action,Crime,Drama"
158583,tt0163944,tvMiniSeries,Labyrinten,Labyrinten,0,2000,2000,259,"Drama,Thriller"
...,...,...,...,...,...,...,...,...,...
9243198,tt9915022,tvSeries,Yarali Kuslar,Yarali Kuslar,0,2019,2019,90,Drama
9243231,tt9915114,tvSeries,Touche pas à mon sport,Touche pas à mon sport,0,2015,2016,65,"Sport,Talk-Show"
9243332,tt9915338,tvSeries,Aunty Donna: Camp Bush Camp!,Aunty Donna: Camp Bush Camp!,0,2018,2018,5,Comedy
9243538,tt9915822,tvSeries,Ichhapyaari Naagin,Ichhapyaari Naagin,0,2016,2017,20,Fantasy


In [13]:
# getting all the movies
is_movie = basics['titleType'].str.contains('Movie', case = False)
basics = basics[is_movie]

basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [14]:
#checking info is consistent
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          0 non-null      object
 1   titleType       0 non-null      object
 2   primaryTitle    0 non-null      object
 3   originalTitle   0 non-null      object
 4   isAdult         0 non-null      object
 5   startYear       0 non-null      int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  0 non-null      object
 8   genres          0 non-null      object
dtypes: int32(1), object(8)
memory usage: 0.0+ bytes


## AKAS Data and Cleaning

In [15]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [16]:
# data dictionary indicates nans replaced with /N, replacing with np.nan
akas.replace({'\\N':np.nan}, inplace = True)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [17]:
akas.isna().sum()

titleId                   0
ordering                  0
title                     5
region              1863557
language            6270091
types              27985091
attributes         33032509
isOriginalTitle        2187
dtype: int64

In [18]:
akas.dropna(inplace = True)
akas.isna().sum()

titleId            0
ordering           0
title              0
region             0
language           0
types              0
attributes         0
isOriginalTitle    0
dtype: int64

In [19]:
akas.duplicated().sum()

0

In [20]:
akas.nunique()

titleId            435
ordering            38
title              451
region               5
language             4
types                3
attributes          38
isOriginalTitle      1
dtype: int64

In [21]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453 entries, 100764 to 32801278
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          453 non-null    object
 1   ordering         453 non-null    int64 
 2   title            453 non-null    object
 3   region           453 non-null    object
 4   language         453 non-null    object
 5   types            453 non-null    object
 6   attributes       453 non-null    object
 7   isOriginalTitle  453 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.9+ KB


In [22]:
#showing only movies made in the use
is_US_Movie = akas['region'].str.contains('US',case=False)
akas = akas[is_US_Movie]
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
100764,tt0022542,1,Di shtime fun Yisroel,US,yi,alternative,YIVO translation,0
112667,tt0024265,4,Geleb un gelakht,US,yi,alternative,modern translation,0
116045,tt0024751,9,Avram Ovenu,US,yi,alternative,YIVO translation,0
124644,tt0026010,3,Der yidishe Kenigen Lir,US,yi,alternative,YIVO translation,0
138075,tt0027911,1,Libe un Laydnshaft,US,yi,alternative,modern translation,0
145670,tt0028902,4,Freylekhe kabtsonim,US,yi,alternative,YIVO translation,0
146144,tt0028957,4,Grine Felder,US,yi,alternative,YIVO translation,0
146598,tt0029013,3,Di heylige Shvue,US,yi,alternative,YIVO translation,0
146792,tt0029042,3,Ikh vil zayn a Mame,US,yi,alternative,YIVO translation,0
152375,tt0029765,3,Vu iz mayn Kind?,US,yi,alternative,YIVO translation,0


In [23]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

Series([], Name: tconst, dtype: bool)

## Ratings Data and Cleaning

In [24]:
# Getting overview of the data
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1910
1,tt0000002,5.8,257
2,tt0000003,6.5,1715
3,tt0000004,5.6,169
4,tt0000005,6.2,2530


In [25]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261448 entries, 0 to 1261447
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1261448 non-null  object 
 1   averageRating  1261448 non-null  float64
 2   numVotes       1261448 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.9+ MB


In [26]:
# data dictionary indicates nans replaced with /N, replacing with np.nan
ratings.replace({'\\N':np.nan}, inplace = True)
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [27]:
# checking duplicates
ratings.duplicated().sum()

0

In [28]:
ratings.describe()

,averageRating,numVotes
count,1.261448e+06,1.261448e+06
mean,6.911323e+00,1.017598e+03
std,1.390662e+00,1.704392e+04
min,1.000000e+00,5.000000e+00
25%,6.200000e+00,1.100000e+01
50%,7.100000e+00,2.500000e+01
75%,7.900000e+00,9.700000e+01
max,1.000000e+01,2.642623e+06


In [29]:
# Filter the basics table down to only include the US by using the filter ratings dataframe
keepers2 =basics['tconst'].isin(ratings['tconst'])
keepers2



Series([], Name: tconst, dtype: bool)

## Saving Data as csv


In [30]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv('Data/title.akas.csv.gz',compression = 'gzip', index=False)
ratings.to_csv('Data/title.ratings.csv.gz', compression = 'gzip', index=False)

# Part 2 

Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

In [31]:
import os, time, json
import tmdbsimple as tmdb 
with open('C:/Users/Sean/.edit/tmdb_api.json', 'r') as f:
    login = json.load(f)
from tqdm.notebook import tqdm_notebook
## Display the keys of the loaded dict
login.keys()
tmdb.API_KEY =  login['api-key']


In [32]:
#saving api call data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 'title.akas.csv.gz',
 'title.ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints']

In [33]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [34]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')




In [35]:
#defining a list of years
YEARS_TO_GET = [2000,2001]

In [36]:
# defining an errors list
errors = [ ]

## Loop Extraction

In [37]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
#Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
#Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()    
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
 #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
 # Retrieve the data for the movie id
            temp = get_movie_with_rating(movie_id)  
 # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
# Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip",
                     index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000: 0it [00:00, ?it/s]

Movies from 2001: 0it [00:00, ?it/s]

In [38]:
#any movie ids that caused an error?
print(f"- Total errors: {len(errors)}") 

- Total errors: 0


Load in your csv.gz's of results for each year extracted.
Concatenate the data into 1 dataframe for the remainder of the analysis.
Once you have your data from the API, they would like you to perform some light EDA to show:
1. How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
* Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
2. How many movies are there in each of the certification categories (G/PG/PG-13/R)?
3. What is the average revenue per certification category?
4. What is the average budget per certification category?

